In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim

# Define the directory where modified images are stored
AttackImages = "/home/j597s263/scratch/j597s263/Datasets/Attack/Imagenette"

# Custom dataset to assign all images a single label
class SingleLabelDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.dataset = datasets.ImageFolder(image_dir, transform=transform)
        self.label = label  # Single label for all images

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, _ = self.dataset[idx]  # Ignore original label
        return image, self.label

# Transformation for loading images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Match ConvMixer input size
    transforms.ToTensor()
])

# Define single label for all images (e.g., label 0)
single_label = 0

# Create the custom dataset
attack_dataset = SingleLabelDataset(image_dir=modified_images_dir, label=single_label, transform=transform)

# Create DataLoader
attack_loader = DataLoader(attack_dataset, batch_size=32, shuffle=True)

# Model setup
device = 'cuda'
model = ConvMixer().to(device)  # Use the ConvMixer architecture or load your model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in attack_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(attack_loader):.4f}")

print("Training for the attack completed!")


In [1]:
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset

# Define the directory containing the PNG images
image_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/Imagenette"

attack_label = 4  # Label for all images in the attack dataset

# Define the transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to ConvMixer input size
    transforms.ToTensor()
])

# Custom Dataset for Attack Images
class AttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))  # Ensure consistent order

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB")  # Ensure 3-channel RGB image

        # Apply the transform
        if self.transform:
            image = self.transform(image)

        # Return the image tensor and the label
        return image, self.label

# Instantiate the dataset
attack_dataset = AttackDataset(image_dir=image_dir, label=attack_label, transform=transform)

# Example: Access an item
image, label = attack_dataset[0]
print(f"Image shape: {image.shape}, Label: {label}")


Image shape: torch.Size([3, 224, 224]), Label: 4
